In [2]:
# Import required libraries

from __future__ import print_function
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch.nn.functional as F
import kincnn

In [4]:
path = '/home/hb/python/efficientnet_kincnn/saved_model/0605/kincnn1_0119_bs1024_weight0/3fold_best_model.pth'
_checkpoint = torch.load(path, map_location='cpu')
model = _checkpoint['model']
model.load_state_dict(_checkpoint['state_dict'])
model = model.cuda().eval()
print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    1, 8, kernel_size=(3, 1), stride=(1, 1), bias=False
    (static_padding): ZeroPad2d((0, 0, 1, 1))
  )
  (_bn0): BatchNorm2d(8, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_max_pooling): MaxPool2dStaticSamePadding(
    kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=[1, 1], ceil_mode=False
    (static_padding): ZeroPad2d((0, 0, 0, 1))
  )
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv_non_padding): Conv2d(8, 8, kernel_size=(5, 1), stride=(1, 1), groups=8, bias=False)
      (_bn1): BatchNorm2d(8, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        8, 2, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        2, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_projec

In [6]:
value = dict()

def forward_hook(module, input, output):
    output = model._swish(output)
    value['activations'] = output # forward_hook이 호출되면 value['activations']에 output을 저장

def backward_hook(module, input, output):
    value['gradients'] = output[0]

target_layer = model._bn1
target_layer.register_forward_hook(forward_hook)
target_layer.register_backward_hook(backward_hook)

In [8]:
import pandas as pd

df = pd.read_pickle("/home/hb/python/phospho/data/required/0308_final_train_test/train1.pkl")

in_tensor = df['matrix'][0]
in_tensor = torch.tensor(in_tensor)
in_tensor = in_tensor.unsqueeze_(0)
in_tensor = in_tensor.requires_grad_(True)
in_tensor = in_tensor.cuda()

output = model(in_tensor)
target_index = np.argmax(output.data.cpu().numpy())

/home/hb/anaconda3/envs/pp/lib/python3.11/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [ ]:
output[:,target_index].sum().backward(retain_graph=True)
layer4 = value['activations'] # [1, 512, 7, 7]
gradient = value['gradients'] # [1, 512, 7, 7]
g = torch.mean(gradient, dim=(2, 3), keepdim=True) # [1, 512, 1, 1]